In [ ]:
!pip install unidecode
!pip install pydot

     |████████████████████████████████| 245kB 2.7MB/s 


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import json
import datetime
import os
import pickle
import warnings
import operator

from google.colab import drive
from unidecode import unidecode
from matplotlib.cm import ScalarMappable
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.image as mpimg
from sklearn.preprocessing import normalize
from sklearn.cluster import Birch

%matplotlib inline

sns.set()

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Manipulação de dados

###1 - Criação de informação sobre dias totais do processo

A função ```datas_df```:

*   Cria uma coluna chamada "data_final", com base na coluna "data_ultimo_mov"
*   Cria uma coluna chamada "tempo_proc_dias", a partir das duas colunas anteriores
* Usa outras três funções auxiliares 



In [ ]:
# Faz a preparação do dataframe, removendo valores e criando colunas
def prep_dias_proc(df):
    # Cria novas colunas com 0
    df['tempo_proc_dias'] = 0
    df['data_ultimo_mov'] = 0

    # Retira linhas onde os movimentos são 0 (não há informação sobre eles)
    df = df[df['movimentoLocal'] != 0]
    df = df[df['movimentoNacional'] != 0]

    # Reseta o índice
    df.reset_index(drop=True, inplace=True)
    return df

# Cria uma coluna chamada "data_ultimo_mov", contendo o movimento mais antigo do dataframe
def coluna_ultimo_mov(df_m):
    # Inicializando a variável de índice e a lista que armazenará os movimentos
    index = 0
    lst_movs_date = []

    # Percorre todas as linhas do df
    for linha in df_m['movimentoLocal']:
        # Verifica se a coluna é uma lista vazia
        if df_m.loc[index, 'movimentoLocal'] != []:
            # Percorre as listas dentro da coluna
            for lista in df_m.loc[index, 'movimentoLocal']:
                # Pega o terceiro valor (data-hora no formato ISO)
                date_mov = lista[2]
                lst_movs_date.append(date_mov)

        # Mesmos passos, só que para a coluna de movimentos nacionais
        if df_m.loc[index, 'movimentoNacional'] != []:
            for lista in df_m.loc[index, 'movimentoNacional']:
                date_mov = lista[1]
                lst_movs_date.append(date_mov)
        
        # Faz sorting da lista de strings. Como está no formato ISO, a ordenação pode ser feita como string
        lst_movs_date = sorted(lst_movs_date)

        # Pega o último elemento da string (o maior valor) e o imputa na coluna de último movimento
        oldest = lst_movs_date[-1]
        df_m.loc[index, 'data_ultimo_mov'] = str(oldest)
        
        # Esvazia a lista e incrementa o índice
        lst_movs_date = []
        index+=1

    return df_m

In [ ]:
# Função principal. Cria o df com a coluna de dias do processo e o retorna.
def datas_df(df_d):
    
    # Chama as funções auxiliares
    df_d = prep_dias_proc(df_d) 
    df_d = coluna_ultimo_mov(df_d)

    index = 0
    
    # Percorre as linhas do df, captando o valor da coluna 'dataAjuizamento'
    for datahora in df_d['dataAjuizamento']:
        # Primeiro movimento: data de ajuizamento
        inic = datahora
        # Último movimento é o de data mais recente
        fim = df_d.loc[index, 'data_ultimo_mov']
        
        
        # Convertendo em datetime para poder realizar operação
        # Se estiver no formato ISO
        if len(inic) == 14:
          d_inicial = datetime.datetime.strptime(inic, '%Y%m%d%H%M%S')
        # Se estiver no formato UNIX Epoch (millisecs desde 1/1/1970)
        elif len(inic) == 12:
          d_inicial = datetime.datetime.strptime(datetime.datetime.utcfromtimestamp(int(int(inic)/1000)).isoformat(), '%Y-%m-%dT%H:%M:%S')
        # Se não for nenhum dos anteriores, eliminar a linha pois ela não contém informação
        # de datas facilmente acessível
        # Obs.: Não resetar o Index devido ao loop, se o Index fosse resetado seriam puladas 
        # tantas linhas quantas fossem removidas, pois a variável 'index' é incrementada
        # em todos os 3 casos
        else:
          lst = df_d[df_d.dataAjuizamento==inic].index.tolist()
          for i in lst:
            df_d.drop(i)
          continue
        d_final = datetime.datetime.strptime(fim, '%Y%m%d%H%M%S')
        
        # Preenchendo a coluna com os dias entre a data final e a data inicial
        df_d.loc[index, 'tempo_proc_dias'] = (d_final - d_inicial).days

        index+=1
    return df_d

### 2 - Total de movimentos

In [ ]:
# Cria as colunas "total_movs_loc", "total_movs_nac" e "total_movs"
def total_movimentos(df):
    df['total_movs_loc'] = np.nan
    df['total_movs_nac'] = np.nan
    df['total_movs'] = np.nan
    index = 0
    
    for i in df['movimentoNacional']:

        try:
            df.loc[index, 'total_movs_nac'] = len(df['movimentoNacional'][index])
        except:
            df.loc[index, 'total_movs_nac'] = 1
        try:
            df.loc[index, 'total_movs_loc'] = len(df['movimentoLocal'][index])
        except:
            df.loc[index, 'total_movs_loc'] = 1
        try:
            df.loc[index, 'total_movs'] = len(df['movimentoLocal'][index]) + len(df['movimentoNacional'][index])
        except:
            df.loc[index, 'total_movs'] = 1
        
        index+=1
    return df

###3 - Alteração coluna grau

In [ ]:
def altera_grau(df):
    filtro_G1 = df['grau'] == 'G1'
    filtro_G2 = df['grau'] == 'G2'
    # Se grau != G1 && grau != G2 == Se !(grau == G1 || grau == G2), pois o grau "G3" está 
    # registrado como "SUP" ou "Sup"
    filtro_G3 = np.logical_not(np.logical_or(filtro_G1, filtro_G2))

    df.loc[filtro_G1, 'grau'] = 1
    df.loc[filtro_G2, 'grau'] = 2
    df.loc[filtro_G3, 'grau'] = 3

    return df

###4 - Colunas movimentos mais comuns

In [ ]:
# Contador de freq de movs nacionais
def cols_movs_comuns(df4, com_desc=True):
    df4[['1_mov_nac_freq', '2_mov_nac_freq']] = 0
    dict_movs = {}
    index = 0

    for linha in df4['movimentoNacional']:
        if df4.loc[index, 'movimentoNacional'] != []:
            for lista in df4.loc[index, 'movimentoNacional']:
                movimento = lista[0]

                if movimento in dict_movs:
                    # Incremento na contagem
                    dict_movs[movimento] += 1
                else:
                    # Recebe o valor 1 (primeira ocorrência)
                    dict_movs[movimento] = 1

            # Pegando o maior valor
            maior = max(dict_movs.items(), key=operator.itemgetter(1))[0]
            df4.loc[index, '1_mov_nac_freq'] = maior

            # Excluindo e fazendo nova verificação de maior valor
            dict_movs.pop(maior, None)
            try:
                maior = max(dict_movs.items(), key=operator.itemgetter(1))[0]
                df4.loc[index, '2_mov_nac_freq'] = maior
            except:
                # Erro ocorre em processos com apenas um movimento nacional
                df4.loc[index, '2_mov_nac_freq'] = 0
        index+=1
        dict_movs = {}
    
    # Cria duas janelas com descrições dos movimentos mais comuns no processo.
    if com_desc:
        df4[['desc_mov1', 'desc_mov2']] = '0'

        # Abre o .csv de movimentos para fins de tradução dos códios
        path = '/content/drive/My Drive/Dados/movimentos.csv'
        df_movimentos = pd.read_csv(path)

        index2 = 0
        for item in df4['1_mov_nac_freq']:
            mov1 = int(df4.loc[index2, '1_mov_nac_freq'])
            mov2 = int(df4.loc[index2, '2_mov_nac_freq'])

            if mov1 != 0:
                try:
                    df4.loc[index2, 'desc_mov1'] = df_movimentos.loc[df_movimentos['codigo']==mov1, 'descricao'].values.item()
                except:
                    df4.loc[index2, 'desc_mov1'] = 0
            if mov2 != 0:
                try:
                    df4.loc[index2, 'desc_mov2'] = df_movimentos.loc[df_movimentos['codigo']==mov1, 'descricao'].values.item()
                except:
                    df4.loc[index2, 'desc_mov2'] = 0

            index2+=1

    return df4

In [ ]:
# Função Wrapper para aplicar as 4 acima em ordem
def deploy(trtnum: str):
  with open('drive/My Drive/Dados/Processos/processos-trt'+trtnum+'.pkl','rb') as f:
    df = pd.read_pickle(f)

  print("Datas")
  df = datas_df(df)
  print("MovimentosTotal")
  df = total_movimentos(df)
  print("Grau")
  df = altera_grau(df)
  print("MovimentosColunas")
  df = cols_movs_comuns(df)
  print("tempoMedioMovimentos")
  df['tempoMedioEntreMovimentos'] = df.apply(lambda row : row['tempo_proc_dias']/row['total_movs'], axis = 1)


  with open('drive/My Drive/Dados/Processos/processos-trt'+trtnum+'.pkl','wb') as f:
    df.to_pickle(f)

Chamando as funções declaradas acima

In [ ]:
for i in range(1,26):
  deploy(str(i))
  print(i)

In [ ]:
# Checando a shape para verificar se as colunas foram criadas corretamente
for i in range(1,26):
    with open('drive/My Drive/Dados/Processos/processos-trt'+str(i)+'.pkl','rb') as f:
        df = pd.read_pickle(f)
    print(i, df.shape)

1 (48029, 29)
2 (45135, 29)
3 (51922, 29)
4 (46315, 29)
5 (41014, 29)
6 (38020, 29)
7 (39596, 29)
8 (42440, 29)
9 (40707, 29)
10 (42336, 29)
11 (34424, 29)
12 (42487, 29)
13 (30338, 29)
14 (27994, 29)
15 (53583, 29)
16 (36741, 29)
17 (41663, 29)
18 (38883, 29)
19 (45137, 29)
20 (33457, 29)
21 (34333, 29)
22 (34553, 29)
23 (2, 29)
24 (33410, 29)
25 (59851, 29)


## Clustering

In [ ]:
# Essa função recebe uma lista de paths contendo o caminho para os .pkl que contém os dados dos TRTs e do TST. 
    # É feita a clusterização e o preenchimento de uma nova coluna, sendo os dados salvos posteriormente

def modelo_cluster_processo(lista_paths, predict=True, threshold=0.35,
                            branching_factor=50,
                            n_clusters=4):
    
    birch = Birch(n_clusters=n_clusters, threshold=threshold,
                  branching_factor=branching_factor)

    # Abre os df processados e executa um partial_fit
    for path in lista_paths:
        with open(path,'rb') as f:
            df = pd.read_pickle(f)

        df2 = df.copy()
        df2 = df[['grau', 'totalAssuntos', 'tempo_proc_dias', 'total_movs_loc', 'total_movs_nac',
           'total_movs', '1_mov_nac_freq', '2_mov_nac_freq', 'tempoMedioEntreMovimentos']]

        # Normaliza o input
        X_norm = normalize(df2.to_numpy(), norm='l2')
        # Treino parcial
        birch.partial_fit(X_norm)
    
    #### Se TRUE, faz predição e coloca em uma nova coluna do DF
    if predict:
        for path in lista_paths:
            with open(path, 'rb') as f:
                df = pd.read_pickle(f)
            
            df2 = df.copy()
            df2 = df[['grau', 'totalAssuntos', 'tempo_proc_dias', 'total_movs_loc', 'total_movs_nac',
           'total_movs', '1_mov_nac_freq', '2_mov_nac_freq', 'tempoMedioEntreMovimentos']]

            X = normalize(df2.to_numpy(), norm='l2')
            
            df['cluster_processo'] = birch.predict(X)
            # Código para salvar o arquivo novamente
            with open(path, 'wb') as f:
                df.to_pickle(f)

In [ ]:
# Essa função recebe um dos dataframes e calcula estatísticas sobre os clusters.
    # o resultado é um .csv para cada dataframe (TRT ou TST), contendo dados sobre os processos em determinado cluster.
    # Vale ressaltar que, dado que os clusters foram feitos com base em todos os dados de todos os TRTs e TST, não é necessário criar
        # um arquivo para cada TRT/TST. Pelo princípio de amostragem, um dos TRTs, que contenha processos em todos os clusters, expressa a realidade de todos.

def cluster_processos_csv(df, trtnum):
    # Cria um índice (clusters de processos)
    index = df['cluster_processo'].unique()
    # Colunas do df de clusters
    lista_colunas = ['media_dias', 'media_movs', 'media_movs_loc', 'media_movs_nac', 'media_assuntos', 
                     'media_dias_entre_movs', 'mov_mais_comum1', 'mov_mais_comum2', 'qtd_processos', 'pontuacao']
    # Criando o df e ordenando o índice 
    cluster_df = pd.DataFrame(index=index, columns=lista_colunas)
    cluster_df.sort_index(inplace=True)

    for cluster in df['cluster_processo'].unique():

        # Preenchendo média de movimentos totais, nacionais e locais
        media_nac = df.loc[df['cluster_processo']==cluster, 'total_movs_nac'].mean()
        media_loc = df.loc[df['cluster_processo']==cluster, 'total_movs_loc'].mean()
        media_tot = df.loc[df['cluster_processo']==cluster, 'total_movs'].mean()
        cluster_df.loc[cluster, 'media_movs_loc'] =  media_loc
        cluster_df.loc[cluster, 'media_movs_nac'] =  media_nac
        cluster_df.loc[cluster, 'media_movs'] =  media_tot

        # Preenchendo média do total de assuntos
        media_assunto = df.loc[df['cluster_processo']==cluster, 'totalAssuntos'].mean()
        cluster_df.loc[cluster, 'media_assuntos'] =  media_assunto

        # Preenchendo média dos dias entre movimentos
        media_dias = df.loc[df['cluster_processo']==cluster, 'tempo_proc_dias'].mean()
        cluster_df.loc[cluster, 'media_dias'] =  media_dias

        # Preenchendo média de dias entre movimentos
        media_entre_movs = df.loc[df['cluster_processo']==cluster, 'tempoMedioEntreMovimentos'].mean()
        cluster_df.loc[cluster, 'media_dias_entre_movs'] =  media_entre_movs

        # Preenchendo movimentos mais comuns naquele cluster
        mov_1 = df.loc[df['cluster_processo']==cluster, '1_mov_nac_freq'].mode()
        mov_2 = df.loc[df['cluster_processo']==cluster, '2_mov_nac_freq'].mode()
        cluster_df.loc[cluster, 'mov_mais_comum1'] =  mov_1[0]
        cluster_df.loc[cluster, 'mov_mais_comum2'] =  mov_2[0]

        # Preenchendo quantidade de processos do cluster
        cluster_df.loc[cluster, 'qtd_processos'] = len(df[df['cluster_processo']==cluster])

    # Pontuação
    lista_dias = list(cluster_df['media_dias'])
    lista_dias_ordenada = sorted(lista_dias)
    pont_dias = []

    lista_diasMovs = list(cluster_df['media_dias_entre_movs'])
    lista_diasMovs_ordenada = sorted(lista_diasMovs)
    pont_dias_movs = []

    lista_movs_nac = list(cluster_df['media_movs_nac'])
    lista_movs_nac_ordenada = sorted(lista_movs_nac, reverse=True)
    pont_movs_nac = []

    lista_qtd_procs = list(cluster_df['qtd_processos'])
    lista_qtd_procs_ord = sorted(lista_qtd_procs, reverse=True)
    pont_qtd_procs = []

    for i in range(len(lista_dias_ordenada)):
        pont_dias.append(len(lista_dias_ordenada)*4 - lista_dias.index(lista_dias_ordenada[i])*4)
        pont_dias_movs.append(len(lista_diasMovs_ordenada) - lista_diasMovs.index(lista_diasMovs_ordenada[i]))
        pont_movs_nac.append(len(lista_movs_nac_ordenada) - lista_movs_nac.index(lista_movs_nac_ordenada[i]))
        pont_qtd_procs.append(len(lista_qtd_procs_ord) - lista_qtd_procs.index(lista_qtd_procs_ord[i]))
        
    l1 = list(map(operator.add, pont_dias, pont_dias_movs))
    l2 = list(map(operator.add, pont_movs_nac, pont_qtd_procs))
    cluster_df['pontuacao'] = list(map(operator.add, l1, l2))
    cluster_df['id_cluster'] = cluster_df.index
    cluster_df.to_csv('drive/My Drive/Dados/Estatísticas Clusters/estatisticas-cluster-processos'+trtnum+'.csv',index=False)

In [ ]:
# Realiza o clustering de unidades judiciais, para o arquivo "orgaos-julgadores-clusterizados.csv"
    # Depende do arquivo orgaos-julgadores-trabalhista.csv, que é feito pelas funções no notebook Manipulação de CSV
   
def modelo_cluster_unidade(path = '/content/drive/My Drive/Dados/orgaos-julgadores-trabalhista.csv',
                            threshold=0.35, branching_factor=50, n_clusters=4,
                            nome_arquivo='/content/drive/My Drive/Dados/orgaos-julgadores-clusterizados.csv'):
    
    birch = Birch(n_clusters=n_clusters, threshold=threshold,
                  branching_factor=branching_factor)
    
    df = pd.read_csv(path)
    df = df[df['qtd_processos'] != 0]
    df2 = df[['CAPITAL', 'qtd_processos', 'tempo_entre_movs', 'cluster_processual_mais_comum']]

    X_norm = normalize(df2.to_numpy(), norm='l2')
    birch.fit(X_norm)

    # Predict
    df['cluster_unidade'] = birch.predict(X_norm)
    df.to_csv(nome_arquivo,index=False)
    

In [ ]:
# Recebe um dataframe de processos de um TRT e gera dados relativos aos processos daquele TRT.
    # essa função precisa ser invocada para todos os arquivos de TRTs/TST
def dados_unidades_judiciais(df):
    unidades_julgadoras = pd.read_csv('/content/drive/My Drive/Dados/orgaos-julgadores-trabalhista.csv')

    # Preenche os dados dos órgãos julgadores com ids nos processos
    for id_orgao in df['codigoOrgaoJulgador'].unique():
        # Quantidade de processos
        qtd_procs = df.loc[df['codigoOrgaoJulgador']==id_orgao].shape[0]
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==id_orgao, 'qtd_processos'] = qtd_procs

        # Tempo médio do processo
        tempo_medio = df.loc[df['codigoOrgaoJulgador']==id_orgao, 'tempo_proc_dias'].mean()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==id_orgao, 'tempo_medio_proc'] = tempo_medio

        # Tempo entre movimentações
        tempo_entre_movs = df.loc[df['codigoOrgaoJulgador']==id_orgao, 'tempoMedioEntreMovimentos'].mean()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==id_orgao, 'tempo_entre_movs'] = tempo_entre_movs

        # Cluster processual mais comum
        cluster_comum = df.loc[df['codigoOrgaoJulgador'] == id_orgao, 'cluster_processo'].mode()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==id_orgao, 'cluster_processual_mais_comum'] = cluster_comum[0]


    # Preenche os dados dos órgãos que são pais dos órgãos julgadores (TRTs e TST)
    lista_pais = list(unidades_julgadoras.loc[unidades_julgadoras['qtd_processos'] != 0, 'SEQ_ORGAO_PAI'].unique())

    for pai in lista_pais:
        # Quantidade de processos
        qtd_procs = unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO_PAI'] == int(pai), 'qtd_processos'].sum()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==int(pai), 'qtd_processos'] = qtd_procs

        # Tempo médio de vida dos processos
        tempo_medio = unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO_PAI'] == int(pai), 'tempo_medio_proc'].mean()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==int(pai), 'tempo_medio_proc'] = tempo_medio

        # Tempo médio entre movimentações 
        tempo_entre_movs = unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO_PAI'] == int(pai), 'tempo_entre_movs'].mean()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==int(pai), 'tempo_entre_movs'] = tempo_entre_movs

        # Cluster processual mais comum
        cluster_comum = unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO_PAI'] == int(pai), 'cluster_processual_mais_comum'].mode()
        unidades_julgadoras.loc[unidades_julgadoras['SEQ_ORGAO']==int(pai), 'cluster_processual_mais_comum'] = cluster_comum[0]

    with open('/content/drive/My Drive/Dados/orgaos-julgadores-trabalhista.csv', 'w') as file:
        unidades_julgadoras.to_csv(file, index=False)

In [ ]:
# Cria um arquivo .csv contendo informações sobre os clusters de unidades judiciais.
    # essa função espera receber um dataframe de orgaos julgadores clusterizados

# DF recebido é o dataframe de orgaos julgadores clusterizados

def cluster_unidades_csv(df):
    # Cria um índice (clusters de processos)
    index = df['cluster_unidade'].unique()
    # Colunas do df de clusters
    lista_colunas = ['media_dias', 'media_dias_entre_movs', 'qtd_media_procs', 'pontuacao_cluster_proc' ,'pontuacao']
    # Criando o df e ordenando o índice 
    cluster_df = pd.DataFrame(index=index, columns=lista_colunas)
    cluster_df.sort_index(inplace=True)

    cluster_procs = pd.read_csv('/content/drive/My Drive/Dados/estatisticas-cluster-processos1.csv')

    for cluster in df['cluster_unidade'].unique():

        # Preenchendo média dos dias
        media_dias = df.loc[df['cluster_unidade']==cluster, 'tempo_medio_proc'].mean()
        cluster_df.loc[cluster, 'media_dias'] =  media_dias

        # Preenchendo média de dias entre movimentos
        media_entre_movs = df.loc[df['cluster_unidade']==cluster, 'tempo_entre_movs'].mean()
        cluster_df.loc[cluster, 'media_dias_entre_movs'] =  media_entre_movs

        # Preenche coluna pontuacao_cluster_proc com a pontuação do cluster de processos mais comum naquela unidade judicial
        pontuacao_proc = cluster_procs.loc[cluster, 'pontuacao']
        cluster_df.loc[cluster, 'pontuacao_cluster_proc'] =  pontuacao_proc

        # Média de processos
        media_procs = df.loc[df['cluster_unidade']==cluster, 'qtd_processos'].mean()
        cluster_df.loc[cluster, 'qtd_media_procs'] = media_procs
    

    # Pontuação
    lista_dias = list(cluster_df['media_dias'])
    lista_dias_ordenada = sorted(lista_dias)
    pont_dias = []

    lista_diasMovs = list(cluster_df['media_dias_entre_movs'])
    lista_diasMovs_ordenada = sorted(lista_diasMovs)
    pont_dias_movs = []

    lista_qtd_procs = list(cluster_df['qtd_media_procs'])
    lista_qtd_procs_ord = sorted(lista_qtd_procs, reverse=True)
    pont_qtd_procs = []
    

    for i in range(len(lista_dias_ordenada)):
        pont_dias.append(len(lista_dias_ordenada)*4 - lista_dias.index(lista_dias_ordenada[i])*4)
        pont_dias_movs.append(len(lista_diasMovs_ordenada) - lista_diasMovs.index(lista_diasMovs_ordenada[i]))
        pont_qtd_procs.append(len(lista_qtd_procs_ord) - lista_qtd_procs.index(lista_qtd_procs_ord[i]))

    l1 = list(map(operator.add, pont_dias, pont_dias_movs))
    l2 = list(cluster_df['pontuacao_cluster_proc'])

    op1 = map(operator.add, l1, l2)

    cluster_df['pontuacao'] = list(map(operator.add, op1, pont_qtd_procs))
    cluster_df['id_cluster'] = cluster_df.index
    cluster_df.to_csv('/content/drive/My Drive/Dados/estatisticas-cluster-unidades.csv', index=False)


Chamando as funções declaradas

In [ ]:
# Cria a lista de paths com todos os arquivos .pkl dos TRTs
lista_paths = []
for file in os.listdir("drive/My Drive/Dados/Processos"):
    if file.endswith(".pkl"):
        lista_paths.append(os.path.join("drive/My Drive/Dados/Processos", file))
lista_paths

['drive/My Drive/Dados/Processos/processos-trt1.pkl',
 'drive/My Drive/Dados/Processos/processos-trt3.pkl',
 'drive/My Drive/Dados/Processos/processos-trt6.pkl',
 'drive/My Drive/Dados/Processos/processos-trt4.pkl',
 'drive/My Drive/Dados/Processos/processos-trt7.pkl',
 'drive/My Drive/Dados/Processos/processos-trt8.pkl',
 'drive/My Drive/Dados/Processos/processos-trt9.pkl',
 'drive/My Drive/Dados/Processos/processos-trt10.pkl',
 'drive/My Drive/Dados/Processos/processos-trt11.pkl',
 'drive/My Drive/Dados/Processos/processos-trt12.pkl',
 'drive/My Drive/Dados/Processos/processos-trt15.pkl',
 'drive/My Drive/Dados/Processos/processos-trt16.pkl',
 'drive/My Drive/Dados/Processos/processos-trt17.pkl',
 'drive/My Drive/Dados/Processos/processos-trt19.pkl',
 'drive/My Drive/Dados/Processos/processos-trt20.pkl',
 'drive/My Drive/Dados/Processos/processos-trt21.pkl',
 'drive/My Drive/Dados/Processos/processos-trt23.pkl',
 'drive/My Drive/Dados/Processos/processos-trt22.pkl',
 'drive/My Drive/

In [ ]:
# Chamando a função que cria o cluster de processos
modelo_cluster_processo(lista_paths)

In [ ]:
# Chamando a função que cria dados sobre os clusters de processos
for i in range(1,26):
    with open('drive/My Drive/Dados/Processos/processos-trt'+str(i)+'.pkl','rb') as f:
        df = pd.read_pickle(f)
    cluster_processos_csv(df, str(i))

In [ ]:
# Verificando o tamanho dos arquivos, apenas para fins de checagem
for path in lista_paths:
    with open(path,'rb') as f:
        df = pd.read_pickle(f)
    print(path, df.shape)

drive/My Drive/Dados/Processos/processos-trt1.pkl (48029, 30)
drive/My Drive/Dados/Processos/processos-trt3.pkl (51922, 30)
drive/My Drive/Dados/Processos/processos-trt4.pkl (46315, 30)
drive/My Drive/Dados/Processos/processos-trt6.pkl (38020, 30)
drive/My Drive/Dados/Processos/processos-trt7.pkl (39596, 30)
drive/My Drive/Dados/Processos/processos-trt8.pkl (42440, 30)
drive/My Drive/Dados/Processos/processos-trt9.pkl (40707, 30)
drive/My Drive/Dados/Processos/processos-trt10.pkl (42336, 30)
drive/My Drive/Dados/Processos/processos-trt11.pkl (34424, 30)
drive/My Drive/Dados/Processos/processos-trt12.pkl (42487, 30)
drive/My Drive/Dados/Processos/processos-trt15.pkl (53583, 30)
drive/My Drive/Dados/Processos/processos-trt16.pkl (36741, 30)
drive/My Drive/Dados/Processos/processos-trt17.pkl (41663, 30)
drive/My Drive/Dados/Processos/processos-trt19.pkl (45137, 30)
drive/My Drive/Dados/Processos/processos-trt20.pkl (33457, 30)
drive/My Drive/Dados/Processos/processos-trt21.pkl (34333, 30)

In [ ]:
# Chamando a função que cria clusters de unidades judiciais
modelo_cluster_unidade(threshold=0.2)

In [ ]:
# Chamando a função que gera dados de unidades judiciais
for path in lista_paths:
    with open(path,'rb') as f:
        df = pd.read_pickle(f)
    dados_unidades_judiciais(df)
    print(path)

drive/My Drive/Dados/Processos/processos-trt1.pkl
drive/My Drive/Dados/Processos/processos-trt3.pkl
drive/My Drive/Dados/Processos/processos-trt6.pkl
drive/My Drive/Dados/Processos/processos-trt4.pkl
drive/My Drive/Dados/Processos/processos-trt7.pkl
drive/My Drive/Dados/Processos/processos-trt8.pkl
drive/My Drive/Dados/Processos/processos-trt9.pkl
drive/My Drive/Dados/Processos/processos-trt10.pkl
drive/My Drive/Dados/Processos/processos-trt11.pkl
drive/My Drive/Dados/Processos/processos-trt12.pkl
drive/My Drive/Dados/Processos/processos-trt15.pkl
drive/My Drive/Dados/Processos/processos-trt16.pkl
drive/My Drive/Dados/Processos/processos-trt17.pkl
drive/My Drive/Dados/Processos/processos-trt19.pkl
drive/My Drive/Dados/Processos/processos-trt20.pkl
drive/My Drive/Dados/Processos/processos-trt21.pkl
drive/My Drive/Dados/Processos/processos-trt23.pkl
drive/My Drive/Dados/Processos/processos-trt22.pkl
drive/My Drive/Dados/Processos/processos-trt5.pkl
drive/My Drive/Dados/Processos/processo

In [ ]:
# Chamando a função que cria dados dos clusters de unidades 
path = '/content/drive/My Drive/Dados/orgaos-julgadores-clusterizados.csv'
df = pd.read_csv(path)
cluster_unidades_csv(df)

## Pontuação de unidades e processos

In [ ]:
# Pontuação do processo (menor = melhor)
def pontuacao_processos():

    # Leitura do .csv de unidades
    unidades = pd.read_csv('/content/drive/My Drive/Dados/orgaos-julgadores-clusterizados.csv')
    
    # Percorrendo todos os arquivos dos TRTs
    for numtrt in range(1, 26):
        index_trt = 0
        with open('drive/My Drive/Dados/Processos/processos-trt'+str(numtrt)+'.pkl','rb') as f:
            df = pd.read_pickle(f)
        # Inicializando a coluna de pontuaçao com 0
        df['pontuacao_processo'] = 0
        print("TRT ", numtrt)
        # Lendo o arquivo de stats do respectivido TRT
        stats = '/content/drive/My Drive/Dados/Estatísticas Clusters/estatisticas-cluster-processos{}.csv'.format(str(numtrt))
        stats_df = pd.read_csv(stats, index_col='id_cluster')
        # Pegando a maior pontuação de cluster
        maior_pontuacao_cluster = max(stats_df['pontuacao'])
        

        # Percorrendo todos os processos do TRT em questão
        index_proc = 0
        for processo in df['numero']:
            # Captando qual é o cluster do processo
            id_cluster = df.loc[index_proc, 'cluster_processo']

            # Calculando a pontuação do cluster
            pontuacao_cluster = pow(maior_pontuacao_cluster, 2)/(stats_df.loc[id_cluster, 'pontuacao'] + 1)
            # Calculando desvio entre o tempo entre movimentos e dias do processo com relação às estatísticas dos processos de unidades similares
            desvio_tempo_entre_movimentos = abs(df.loc[index_proc, 'tempoMedioEntreMovimentos']) - abs(stats_df.loc[id_cluster, 'media_dias_entre_movs'])
            desvio_tempo_total = abs(df.loc[index_proc, 'tempo_proc_dias']) - abs(stats_df.loc[id_cluster, 'media_dias'])
            
            # Calculando a pontuação do processo
            pontuacao = pontuacao_cluster + desvio_tempo_entre_movimentos/pontuacao_cluster + desvio_tempo_total/pontuacao_cluster
            # Atribuindo a pontuação
            df.loc[index_proc, 'pontuacao_processo'] = pontuacao
            
            index_proc+=1 
        
        # Calculando qual o valor de pontuação dos maiores 10% (no caso, os 10% mais mal avaliados) 
        quantile = df['pontuacao_processo'].quantile(.9)
        # Se for a primeira iteração, esse valor é atribuído a um df novo. Caso contrário, é anexado ao df.
        if index_trt == 0:
            df_alerta = df.loc[df['pontuacao_processo']>quantile]
            df_alerta.reset_index(drop=True, inplace=True)
        else:
            b = df.loc[df['pontuacao_processo']>quantile]
            df_alerta = pd.concat([df_alerta, b], ignore_index=True)
            df_alerta.reset_index(drop=True, inplace=True)
        
        # Preenchendo a quantidade de processos em alerta para cada unidade 
        lst_pai = []
        # Percorrendo uma lista contendo os valores únicos de órgão julgador para cada TRT
        for id_orgao in df_alerta['codigoOrgaoJulgador'].unique():
            # Se o valor for 0, continuar
            if id==0:
                continue
            
            # Captando a quantidade de processos em alerta em dado órgão de código id_orgao
            qtd_processos_orgao = df_alerta[df_alerta['codigoOrgaoJulgador']==id_orgao].shape[0]
            # Anexando no df de unidades
            unidades.loc[unidades['SEQ_ORGAO']==id_orgao, 'procs_em_alerta'] = qtd_processos_orgao
            
            # Capturando informação do pai do órgão julgador
            try:
                pai = int(unidades.loc[unidades['SEQ_ORGAO']==id_orgao, 'SEQ_ORGAO_PAI'])
            except:
                continue
            if pai in lst_pai:
                continue
            else:
                lst_pai.append(pai)
        # Percorrendo a lista de pais
        for pai in lst_pai:
            # Anexando o total de processos de seus filhos
            qtd_pai = unidades.loc[unidades['SEQ_ORGAO_PAI'] == int(pai), 'procs_em_alerta'].sum()
            unidades.loc[unidades['SEQ_ORGAO']==pai, 'procs_em_alerta'] == qtd_pai

        # Salvando o df como .pkl
        with open('drive/My Drive/Dados/Processos/processos-trt'+str(numtrt)+'.pkl','wb') as f:
            df.to_pickle(f)
        
        # Incrementando o índice
        index_trt+=1
    
    # Salvando o dataframe que contém todos os processos em alerta
    df_alerta.to_csv('/content/drive/My Drive/Dados/processos-em-alerta.csv', index=False)
    # Salvando a nova versão do arquivo de unidades judiciais
    unidades.to_csv('/content/drive/My Drive/Dados/orgaos-julgadores-clusterizados.csv', index=False)

In [ ]:
pontuacao_processos()

TRT  1
TRT  2
TRT  3
TRT  4
TRT  5
TRT  6
TRT  7
TRT  8
TRT  9
TRT  10
TRT  11
TRT  12
TRT  13
TRT  14
TRT  15
TRT  16
TRT  17
TRT  18
TRT  19
TRT  20
TRT  21
TRT  22
TRT  23
TRT  24
TRT  25


In [ ]:
### Gera as pontuações das unidades

# Leitura do arquivo que contém informações sobre as unidades
unidades = pd.read_csv('/content/drive/My Drive/Dados/orgaos-julgadores-clusterizados.csv')
unidades.to_csv('/content/drive/My Drive/Dados/orgaos-julgadores-clusterizados.csv', index=False)
# Leitura do arquivo que contém informações sobre o cluster de unidades
cluster_unidades = pd.read_csv('/content/drive/My Drive/Dados/estatisticas-cluster-unidades.csv', index_col='id_cluster')

# Inicializando o índice, criando uma coluna e captando o maior valor de pontuação de clusters de unidades
index = 0
unidades['pontuacao_unidade'] = 0
maior_pontuacao_cluster = max(cluster_unidades['pontuacao'])

# Percorrendo todas as linhas do dataframe de unidades
for row in unidades['NOMEDAVARA']:
    # Captando a informação sobre o cluster da unidade
    id_cluster = unidades.loc[index, 'cluster_unidade']

    # Gerando dados
    qtd_processos_alerta = unidades.loc[index, 'procs_em_alerta'] # quantidade de processos em alerta. Menor = melhor
    pont_cluster = pow(maior_pontuacao_cluster, 2)/(cluster_unidades.loc[id_cluster, 'pontuacao'] + 1) # Pontuação do cluster, manipulada para: menor = melhor
    # Desvio entre o tempo entre movimentos da unidade e o tempo entre movimentos do cluster
    desvio_tempo_entre_movimentos = abs(unidades.loc[index, 'tempo_entre_movs']) - abs(cluster_unidades.loc[id_cluster, 'media_dias_entre_movs']) 
    # Desvio entre o tempo médio total da unidade e o tempo total médio do cluster
    desvio_tempo_total = abs(unidades.loc[index, 'tempo_medio_proc']) - abs(cluster_unidades.loc[id_cluster, 'media_dias'])

    # Relação entre a quantidade de processos e a quantidade de processos em alerta (0 -> não há processos em alerta, 1 -> todos os processos estão em alerta)
    try:
        procs_total_alerta = unidades.loc[index, 'qtd_processos']/(qtd_processos_alerta)
    except:
        procs_total_alerta = 0
    
    # Calculando a pontuação e a atribuindo à respectiva coluna 
    pontuacao = qtd_processos_alerta + pont_cluster + (desvio_tempo_entre_movimentos + desvio_tempo_total)*procs_total_alerta
    unidades.loc[index, 'pontuacao_unidade'] = pontuacao
    index+=1

In [ ]:
### Gerando arquivos de ranking
# Ranking 1º Grau
varas_t = unidades.loc[unidades['GRAU_ORGAO']=='GRA1T']
varas_t['ranking'] = varas_t['pontuacao_unidade'].rank(method='min')
varas_t.set_index('ranking', drop=False, inplace=True)
varas_t.sort_index(inplace=True)
varas_t.to_csv('/content/drive/My Drive/Dados/ranking-1grau.csv', index=True)

# Ranking 2º Grau
gabs_t = unidades.loc[unidades['GRAU_ORGAO']=='GRA2T']
gabs_t['ranking'] = gabs_t['pontuacao_unidade'].rank(method='min')
gabs_t.set_index('ranking', drop=False, inplace=True)
gabs_t.sort_index(inplace=True)
gabs_t.to_csv('/content/drive/My Drive/Dados/ranking-2grau.csv', index=True)

# Ranking de Tribunais Federais
trts = unidades.loc[unidades['GRAU_ORGAO']=='TRIBT']
trts['ranking'] = trts['pontuacao_unidade'].rank(method='min')
trts.set_index('ranking', drop=False, inplace=True)
trts.sort_index(inplace=True)
trts.to_csv('/content/drive/My Drive/Dados/ranking-trts.csv', index=True)

# Ranking de Gabinetes de Ministros 3º Grau
gabim = unidades.loc[unidades['GRAU_ORGAO']=='GABIM']
gabim['ranking'] = gabim['pontuacao_unidade'].rank(method='min')
gabim.set_index('ranking', drop=False, inplace=True)
gabim.sort_index(inplace=True)
gabim.to_csv('/content/drive/My Drive/Dados/ranking-3grau.csv', index=True)

# Ranking Geral
geral = unidades.copy()
geral['ranking'] = geral['pontuacao_unidade'].rank(method='min')
geral.set_index('ranking', drop=False, inplace=True)
geral.sort_index(inplace=True)
geral.to_csv('/content/drive/My Drive/Dados/ranking-geral.csv', index=True)